<a href="https://colab.research.google.com/github/michael-wettach/pythonsamples/blob/main/XMLsamples/xml_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Jetzt wollen wir ein XML validieren.

In [3]:
from lxml import etree
from io import StringIO

# open and read schema file
with open('FinS_XSD_Light_extended_19052021.xsd', 'r') as schema_file:
    # parse the schema (assume well-formed schema, skip error checking)
    xmlschema_doc = etree.parse(schema_file)
    xmlschema = etree.XMLSchema(xmlschema_doc)
    print("XML schema was parsed.")

# open and read xml file
with open('dansource_FISTAPB_2_202105201559037326.000001.xml', 'r') as xml_file:
    # parse the XML document (check for XML syntax before validation)
    try:
        doc = etree.parse(xml_file)
        print('XML well formed, syntax ok.')

    # check for file IO error
    except IOError:
        print('Invalid File')
        raise

    # check for XML syntax errors
    except etree.XMLSyntaxError as err:
        print('XML Syntax Error, see error_syntax.log')
        with open('error_syntax.log', 'w') as error_log_file:
            error_log_file.write(str(err.error_log))
        raise

    except:
        print('Unknown error in parsing XML, exiting.')
        raise

# now that doc parsed successfully, validate against schema
try:
    xmlschema.assertValid(doc)
    print('XML valid, schema validation ok.')

except etree.DocumentInvalid as err:
    print('Schema validation error, see error_schema.log')
    with open('error_schema.log', 'w') as error_log_file:
        error_log_file.write(str(err.error_log))
    raise

except:
    print('Unknown error in validating XML, exiting.')
    raise


XMLSchemaParseError: ignored

In [8]:
# Neuer Versuch mit easyxsd
!pip install https://github.com/gnrfan/python-easyxsd/zipball/master

  Using cached https://github.com/gnrfan/python-easyxsd/zipball/master
  Created wheel for easyxsd: filename=easyxsd-0.1-cp37-none-any.whl size=4583 sha256=31a59267884e85afbc226b8674fc2312523a2c2c8b9b491d2c8ee2d28685751c
  Stored in directory: /tmp/pip-ephem-wheel-cache-zf5p_he0/wheels/81/c9/b8/475f289cd991a37b25f01593b8027463abf7459ca7e10132a0
Successfully built easyxsd


In [12]:
from easyxsd import *
# Load XML Schema (.xsd file)
xsd = xsd_from_file("FinS_XSD_Light_extended_19052021.xsd")

# Load XML File
xml = xml_from_file("dansource_FISTAPB_2_202105201559037326.000001.xml")

# Validate
list_errors = validate_with_errors(xml, xsd)
print(*list_errors)

XMLSchemaParseError: ignored

In [1]:
# Neuer Versuch mit xmlschema
!pip install xmlschema

     |████████████████████████████████| 256kB 4.0MB/s 
     |████████████████████████████████| 153kB 20.1MB/s 


In [2]:
import xmlschema
my_schema = xmlschema.XMLSchema('FinS_XSD_Light_extended_19052021.xsd')
my_schema.validate('dansource_FISTAPB_2_202105201559037326.000001.xml')

XMLSchemaModelError: ignored

Aus der Fehlermeldung geht klar hervor: sie bezieht sich auf den vierten Knoten vom Typ Element: element[4].
1. element name = "docs"
2. element name = "doc" (Wiederholung)
3. element name = "header"
4. element name = "payload"

In diesem 4. Element "payload" betrifft der Befund gemäß Path-Angabe im complexType die sequence. Innerhalb der Sequence gibt es drei Elemente mit Namen "elements", die der Parser für nicht eindeutig hält:
- KAZInfo (habe ich auf occurs 1,1 gesetzt)
- GiroRecord (habe ich auf occurs 1,unbounded gesetzt)
- TagesgeldRecord (habe ich auf occurs 0,unbounded gesetzt)

Die Meldung hat mit der eigentlichen XML Datei nichts zu tun, sie tritt bereits bei der Prüfung des xsd Schemas auf. Aufgrund der Occurs-Angaben ist klar, dass hier zunächst KAZInfo und GiroRecord gemeint sind.

Was ist nun das Problem? Die Meldung sagt:

XMLSchemaModelError: Element Declarations Consistent violation between XsdElement(name='elements', occurs=[1, None]) and XsdElement(name='elements', occurs=[1, 1]): match the same name but with different types

https://www.w3.org/TR/2004/REC-xmlschema-1-20041028/structures.html#cos-element-consistent 
Die XML Regel <b>Schema Component Constraint / Element Declarations Consistent</b>:<br/>
If the {particles} contains, either directly, indirectly (that is, within the {particles} of a contained model group, recursively) or ·implicitly· two or more element declaration particles with the same {name} and {target namespace}, then all their type definitions must be the same top-level definition, that is, all of the following must be true:
- all their {type definition}s must have a non-·absent· {name}.
- all their {type definition}s must have the same {name}.
- all their {type definition}s must have the same {target namespace}.

Auf Stackoverflow habe ich dazu folgende Hinweise gefunden:
- You can't declare two elements with the same name with different types in the same context.
- If your data is similar, and the main difference is an attribute which describes the text content of the element, you can create one type and restrict the values the attribute can receive
- If your content is not similar you have to create two types (and it would also make sense for them to have different names or to at least occur in another context). 

Zum Context siehe https://wiki.scn.sap.com/wiki/display/XI/What+is+Context+and+Context+Change.

Lösungsideen gemäß https://stackoverflow.com/questions/827051/xml-schema-for-sequence-of-elements-with-same-name-but-different-attribute-value wären daher:
- Für jedes Element einen eigenen Type definieren, ggf. in Verbindung mit 
Validierung durch xmllint, das scheint weniger streng zu sein
- Validierung mit XML Schema 1.1, da gibt es diese Einschränkung nicht


Ich habe zum Vergleich noch mit dem OpenSource xsd-Editor (https://sourceforge.net/projects/xsdeditor/) validiert und diesen Befund bekommen: 

complex type '__AnonC99' violates the unique particle attribution rule in its components 'elements' and 'elements'

Das bezieht sich wohl auf dieselbe Stelle, meldet aber eine andere Regelverletzung.

Die XML Regel <b>Schema Component Constraint / Unique Particle Attribution</b>:
A content model must be formed such that during ·validation· of an element information item sequence, the particle component contained directly, indirectly or ·implicitly· therein with which to attempt to ·validate· each item in the sequence in turn can be uniquely determined without examining the content or attributes of that item, and without any information about the items in the remainder of the sequence. 

Das heißt, dass der Parser nicht eindeutig entscheiden kann, mit welchem Knoten mit Name = "elements" er eine Zeile parsen soll. Für diese Unterscheidung hätten wir eigentlich das <i>Attribut</i> "name", aber die Regel besagt ja, die Zuordnung muss möglich sein "without examining the attributes of that item". Bei den Knoten mit Name = "element" bekommt er das aber offenbar trotzdem hin, die heißen ja auch alle gleich. Daher bin ich skeptisch, ob es tatsächlich diese Regel hier ist, die das Problem macht.  
